<a href="https://colab.research.google.com/github/devbijoy274/ai_Implementation/blob/master/review_form_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **An auto perfomance evaluator**

In [0]:
class EvaluateFeedback:
  def __init__(self, data):
    self.data = data
    
  def get_ratings_from_feedback(self, feedback):
    sentences = feedback.split("\n")
    total_rating = 0;
    comp_sentence = ""
    for sent in sentences:
      comp_sentence += sent
    feedback_sentence = ""
    words = self.word_tok.tokenize(comp_sentence)
    for val in words:
      feedback_sentence += val + " "
    feedback_sentence = feedback_sentence.rstrip()
    blob = TextBlob(feedback_sentence)
    rating = blob.sentiment.polarity
    return rating
  
  def get_rate_scale(self, value):
    rating = 1;
    if (value <= 0):
      rating = 1
    elif(value > 0 and value <= 0.4):
      rating = 2
    elif(value > 0.4 and value <= 0.6):
      rating = 3
    elif(value > 0.6):
      rating = 4
    return rating 
  
  def get_feedback(self):
    accomplishment = self.data.iloc[7]
    ratings_classes = ['Needs Improvement', 'Meets Expectations', 'Consistently Exceeds Expectations', 'Occasionally Exceeds Expectations']
    feedback_classes = ['Negative', 'good', 'very good', 'excellent']

    is_mgr2_not_present = pd.isnull(data.iloc[3,5])

    accomplishment_mgr1_rating = ratings_classes.index(accomplishment[5]) + 1
    accomplishment_mgr1_feedback = accomplishment[4]
    
    self.word_tok = RegexpTokenizer(r'\w+')
    num_tok = RegexpTokenizer(r'\D+')
    mgr1_feedback_evaluated = EvaluateFeedback.get_ratings_from_feedback(self, accomplishment_mgr1_feedback)
    mgr1_feedback_rate_scale = EvaluateFeedback.get_rate_scale(self, mgr1_feedback_evaluated)
    
    contributions = data.iloc[8]
    contributions_mgr1_rating = ratings_classes.index(contributions[5]) + 1
    contributions_mgr1_feedback = contributions[4]
    
    mgr1_feedback_evaluated_v2 = EvaluateFeedback.get_ratings_from_feedback(self, contributions_mgr1_feedback)
    mgr1_feedback_rate_scale_v2 = EvaluateFeedback.get_rate_scale(self,mgr1_feedback_evaluated_v2)
    mg1_overall_rating = math.ceil((accomplishment_mgr1_rating + contributions_mgr1_rating ) / 2)
    mg1_overall_feedback = (mgr1_feedback_rate_scale + mgr1_feedback_rate_scale_v2) / 2

    if (not(is_mgr2_not_present)):      
      contributions_mgr2_rating = ratings_classes.index(contributions[7]) + 1
      contributions_mgr2_feedback = contributions[6]
      
      mgr2_feedback_evaluated_v2 = EvaluateFeedback.get_ratings_from_feedback(self, contributions_mgr2_feedback)
      mgr2_feedback_rate_scale_v2 = EvaluateFeedback.get_rate_scale(self, mgr2_feedback_evaluated_v2)

      accomplishment_mgr2_rating = ratings_classes.index(accomplishment[7]) + 1
      accomplishment_mgr2_feedback = accomplishment[6]
      mgr2_feedback_evaluated = EvaluateFeedback.get_ratings_from_feedback(self, accomplishment_mgr2_feedback)
      mgr2_feedback_rate_scale = EvaluateFeedback.get_rate_scale(self, mgr2_feedback_evaluated)

      mg2_overall_rating = math.ceil((accomplishment_mgr2_rating + contributions_mgr2_rating) / 2)
      mg2_overall_feedback = (mgr2_feedback_rate_scale + mgr2_feedback_rate_scale_v2) / 2

    if(is_mgr2_not_present):
      overall_emp_rating = mg1_overall_rating
      overall_emp_feedback = mg1_overall_feedback
      return [ratings_classes[int(overall_emp_rating) - 1], feedback_classes[int(overall_emp_feedback) - 1]]
    else:
      overall_emp_rating = (mg1_overall_rating + mg2_overall_rating)/2
      overall_emp_feedback = (mg1_overall_feedback + mg2_overall_feedback)/2
      return [ratings_classes[int(overall_emp_rating) - 1], feedback_classes[int(overall_emp_feedback) - 1]]

In [2]:
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import numpy as np
from textblob import TextBlob
import nltk
import math
import pdb

nltk.download('punkt')

year = "2018_19"

emp_no = ['MSI1', 'MSI2', 'MSI3', 'MSI4', 'MSI5']
emp_name = ['abc', 'def', 'ghi', 'jkl', 'mno']

emp_ratings, emp_feedback = [], []

for i in range(len(emp_no)):
  employee_doc = emp_no[i] + "_" + emp_name[i] + "_" + year+".xlsx"
  data = pd.read_excel(employee_doc)
  emp_ratings.append(EvaluateFeedback(data).get_feedback()[0])
  emp_feedback.append(EvaluateFeedback(data).get_feedback()[1])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['Meets Expectations',
 'Meets Expectations',
 'Meets Expectations',
 'Meets Expectations',
 'Meets Expectations']

In [4]:
df = pd.DataFrame({'Emp_ID': emp_no,
                  'Emp_Name': emp_name,
                  'Emp_Rating': emp_ratings,
                  'Emp_Feedback': emp_feedback})
df

,Emp_ID,Emp_Name,Emp_Rating,Emp_Feedback
0,1,abc,Meets Expectations,very good
1,2,def,Meets Expectations,very good
2,3,ghi,Meets Expectations,very good
3,4,jkl,Meets Expectations,very good
4,5,mno,Meets Expectations,very good


In [0]:
# Run this if you want to download the report as csv file format

from google.colab import files

df.to_csv('review_form.csv',index=False)
files.download('review_form.csv')
